In [1]:
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import pandas as pd
import shutil
import time

In [2]:
# define folder paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Average AAs')

# open rgi df
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
# get list of all the .tif AAs in the folder
all_tifs_names = os.listdir(folder_products)

# keep only tif file paths
all_tifs_names = [i for i in all_tifs_names if i[-3:]=='tif']

# make them into full paths
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]

# get rgiid
all_rgi_id = [i[3:17] for i in all_tifs_names]

# make df
df = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths})

# add o2, o3 region
df = df.merge(rgi_gdf, on='RGIId', how='left')
print(len(df))
df.head()
df['O2Region'].unique()

2880


array(['2', '6', '4', '3', '5'], dtype=object)

In [11]:
paths = []
for n in [2,3,4,5,6]:
    print(n)
#     if n!=5: continue

    # keep only the rgi ids that are in o2region x
    df_subset = df[df['O2Region']==str(n)]
    
    # grab the full list of paths
    all_paths = df_subset['tif_path'].values

    # define path for vrt, output location/name
    out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', f"Region_0{n}_AA.tif")
    vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', f'merged{n}.vrt')

    # build vrt dataset
    vrt = gdal.BuildVRT(vrt_path, list(all_paths), VRTNodata='nan', srcNodata=0)    
    vrt = 0
#     # Open the VRT dataset
#     ds = gdal.Open(vrt_path)
    
#     # pause for a few seconds to make sure the vrt gets built
#     time.sleep(180)
    
#     print(f"Mosaicing Region {n}")
#     gdal.Translate(out_path, ds, outputType=gdal.GDT_Byte, creationOptions = ['PREDICTOR=2','COMPRESS=DEFLATE', 'ZLEVEL=9'])
# #     del vrt, ds
# #     os.remove(vrt_path)
#     print("mosaicing done")
print("Done")

2
3
4
5
6
Done


In [12]:
for n in [2,3,4,5,6]:
    print(n)
#     if n!=5: continue

#     # keep only the rgi ids that are in o2region x
#     df_subset = df[df['O2Region']==str(n)]
    
#     # grab the full list of paths
#     all_paths = df_subset['tif_path'].values

    # define path for vrt, output location/name
    out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', f"Region_0{n}_AA.tif")
    vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', f'merged{n}.vrt')

#     # build vrt dataset
#     vrt = gdal.BuildVRT(vrt_path, list(all_paths), VRTNodata='nan', srcNodata=0)    
    
    # Open the VRT dataset
    ds = gdal.Open(vrt_path)
    
    # pause for a few seconds to make sure the vrt gets built
#     time.sleep(180)
    
    print(f"Mosaicing Region {n}")
    gdal.Translate(out_path, ds, outputType=gdal.GDT_Byte, creationOptions = ['PREDICTOR=2','COMPRESS=DEFLATE', 'ZLEVEL=9'])
#     del vrt, ds
#     os.remove(vrt_path)
    print("mosaicing done")

2
Mosaicing Region 2
mosaicing done
3
Mosaicing Region 3
mosaicing done
4
Mosaicing Region 4
mosaicing done
5
Mosaicing Region 5
mosaicing done
6
Mosaicing Region 6
mosaicing done
